In [17]:
indices_array = ['BANKNIFTY', 'NIFTY', 'FINNIFTY', 'MIDCPNIFTY', 'SENSEX']
indices_lot_size = [15, 50, 40, 75, 10]
changes_in_strike = [100, 50, 50, 25, 100]
sensex_expiry_date = '2024-05-03'

In [18]:
from NorenRestApiPy.NorenApi import  NorenApi

class ShoonyaApiPy(NorenApi):
    def __init__(self):
        NorenApi.__init__(self, host='https://api.shoonya.com/NorenWClientTP/', websocket='wss://api.shoonya.com/NorenWSTP/')        
        global api
        api = self

In [19]:
import pyotp

api = ShoonyaApiPy()

In [20]:
id = 4
lots = 2
lower_limit = 71900.00
upper_limit = 77000.00

index = indices_array[id]
lot_size = indices_lot_size[id]

In [21]:
user = "FA267228"
pwd = "Meet@1811"
vc = "FA267228_U"
app_key = "6ebfa36708493ccaed011fdd2ebe2298"
token = "Q7USK6X664V5544L5NB45HM3N7UU5K4A"

In [23]:
api = ShoonyaApiPy()
ret = api.login(userid=user, password=pwd, twoFA=pyotp.TOTP(token).now(), vendor_code=vc, api_secret=app_key, imei='abc1234')
print(ret)
with open('session_token.txt','w') as file:
    file.write(ret['susertoken'])


None


TypeError: 'NoneType' object is not subscriptable

In [11]:
# ret = api.login(userid=user, password=pwd, twoFA=pyotp.TOTP(token).now(), vendor_code=vc, api_secret=app_key, imei='abc1234')
session_token = open('session_token.txt','r').read()
print(len(session_token))
api.set_session(user, pwd, session_token)
api.get_limits()

65


{'stat': 'Not_Ok', 'emsg': 'Session Expired :  Invalid Session Key'}

In [8]:
import pandas as pd
from datetime import datetime,date
nfo_url = 'https://api.shoonya.com/NFO_symbols.txt.zip'
bfo_url = 'https://api.shoonya.com/BFO_symbols.txt.zip'
symbolDf = pd.read_csv(bfo_url if index == 'SENSEX' else nfo_url)
symbolDf['Expiry'] = pd.to_datetime(symbolDf['Expiry']).apply(lambda x: x.date())   
symbolDf

,Exchange,Token,LotSize,Symbol,TradingSymbol,Expiry,Instrument,OptionType,StrikePrice,TickSize,Unnamed: 10
0,BFO,862590,3000,ZEELOPT,ZEEL24MAY195PE,2024-05-30,OPTSTK,PE,195.0,0.05,NaN
1,BFO,862428,3000,ZEELOPT,ZEEL24MAY190PE,2024-05-30,OPTSTK,PE,190.0,0.05,NaN
2,BFO,864006,3000,ZEELOPT,ZEEL24MAY185PE,2024-05-30,OPTSTK,PE,185.0,0.05,NaN
3,BFO,863849,3000,ZEELOPT,ZEEL24MAY180PE,2024-05-30,OPTSTK,PE,180.0,0.05,NaN
4,BFO,865414,3000,ZEELOPT,ZEEL24MAY175PE,2024-05-30,OPTSTK,PE,175.0,0.05,NaN
...,...,...,...,...,...,...,...,...,...,...,...
37875,BFO,2011080,125,ABBLOPT,ABB24APR4200CE,2024-04-25,OPTSTK,CE,4200.0,0.05,NaN
37876,BFO,2042441,125,ABBLOPT,ABB24APR4150CE,2024-04-25,OPTSTK,CE,4150.0,0.05,NaN
37877,BFO,2019451,125,ABBLOPT,ABB24APR4100CE,2024-04-25,OPTSTK,CE,4100.0,0.05,NaN
37878,BFO,2051002,125,ABBLOPT,ABB24APR4050CE,2024-04-25,OPTSTK,CE,4050.0,0.05,NaN


In [101]:
def get_expiry_dates(exchange, symbol):
    import re
    import datetime
    sd = api.searchscrip(exchange, symbol)
    sd = (sd['values'])
    tsym_values = [Symbol['tsym'] for Symbol in sd]
    dates = [re.search(r'\d+[A-Z]{3}\d+', tsym).group() for tsym in tsym_values]
    formatted_dates = [datetime.datetime.strptime(date, '%d%b%y').strftime('%Y-%m-%d') for date in dates]
    sorted_formatted_dates = sorted(formatted_dates)
    expiry_dates = sorted_formatted_dates
#     sorted_dates = [datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%d%b%y').upper() for date in sorted_formatted_dates]
#     expiry_dates = sorted_dates
    return expiry_dates

expiry_date = sensex_expiry_date if index == 'SENSEX' else get_expiry_dates('BFO' if index == 'SENSEX' else 'NFO', index)[0]
print(expiry_date)

2024-04-19


In [102]:
date_object = datetime.strptime(expiry_date, "%Y-%m-%d").date()

ocdf = symbolDf[(symbolDf.Symbol == ('BSXOPT' if index == 'SENSEX' else index)) & (symbolDf.Expiry == date_object)]
ocdf

,Exchange,Token,LotSize,Symbol,TradingSymbol,Expiry,Instrument,OptionType,StrikePrice,TickSize,Unnamed: 10
29206,BFO,881010,10,BSXOPT,SENSEX2441978000PE,2024-04-19,OPTIDX,PE,78000.0,0.05,NaN
29207,BFO,880851,10,BSXOPT,SENSEX2441977900PE,2024-04-19,OPTIDX,PE,77900.0,0.05,NaN
29208,BFO,880918,10,BSXOPT,SENSEX2441977800PE,2024-04-19,OPTIDX,PE,77800.0,0.05,NaN
29209,BFO,880391,10,BSXOPT,SENSEX2441977700PE,2024-04-19,OPTIDX,PE,77700.0,0.05,NaN
29210,BFO,880475,10,BSXOPT,SENSEX2441977600PE,2024-04-19,OPTIDX,PE,77600.0,0.05,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29383,BFO,869973,10,BSXOPT,SENSEX2441969400CE,2024-04-19,OPTIDX,CE,69400.0,0.05,NaN
29384,BFO,870099,10,BSXOPT,SENSEX2441969300CE,2024-04-19,OPTIDX,CE,69300.0,0.05,NaN
29385,BFO,869959,10,BSXOPT,SENSEX2441969200CE,2024-04-19,OPTIDX,CE,69200.0,0.05,NaN
29386,BFO,870089,10,BSXOPT,SENSEX2441969100CE,2024-04-19,OPTIDX,CE,69100.0,0.05,NaN


In [103]:
len(ocdf)

182

In [104]:
Cedf =ocdf[(ocdf.OptionType == 'CE')& (ocdf['StrikePrice'] > lower_limit) & (ocdf['StrikePrice'] < upper_limit)] 
Cedf

,Exchange,Token,LotSize,Symbol,TradingSymbol,Expiry,Instrument,OptionType,StrikePrice,TickSize,Unnamed: 10
29308,BFO,867744,10,BSXOPT,SENSEX2441976900CE,2024-04-19,OPTIDX,CE,76900.0,0.05,NaN
29309,BFO,867648,10,BSXOPT,SENSEX2441976800CE,2024-04-19,OPTIDX,CE,76800.0,0.05,NaN
29310,BFO,867044,10,BSXOPT,SENSEX2441976700CE,2024-04-19,OPTIDX,CE,76700.0,0.05,NaN
29311,BFO,867133,10,BSXOPT,SENSEX2441976600CE,2024-04-19,OPTIDX,CE,76600.0,0.05,NaN
29312,BFO,866911,10,BSXOPT,SENSEX2441976500CE,2024-04-19,OPTIDX,CE,76500.0,0.05,NaN
29313,BFO,866991,10,BSXOPT,SENSEX2441976400CE,2024-04-19,OPTIDX,CE,76400.0,0.05,NaN
29314,BFO,866796,10,BSXOPT,SENSEX2441976300CE,2024-04-19,OPTIDX,CE,76300.0,0.05,NaN
29315,BFO,867245,10,BSXOPT,SENSEX2441976200CE,2024-04-19,OPTIDX,CE,76200.0,0.05,NaN
29316,BFO,867323,10,BSXOPT,SENSEX2441976100CE,2024-04-19,OPTIDX,CE,76100.0,0.05,NaN
29317,BFO,867096,10,BSXOPT,SENSEX2441976000CE,2024-04-19,OPTIDX,CE,76000.0,0.05,NaN


In [105]:
Pedf =ocdf[(ocdf.OptionType == 'PE') & (ocdf['StrikePrice'] > lower_limit) & (ocdf['StrikePrice'] < upper_limit)] 
Pedf

,Exchange,Token,LotSize,Symbol,TradingSymbol,Expiry,Instrument,OptionType,StrikePrice,TickSize,Unnamed: 10
29217,BFO,867942,10,BSXOPT,SENSEX2441976900PE,2024-04-19,OPTIDX,PE,76900.0,0.05,NaN
29218,BFO,867583,10,BSXOPT,SENSEX2441976800PE,2024-04-19,OPTIDX,PE,76800.0,0.05,NaN
29219,BFO,867016,10,BSXOPT,SENSEX2441976700PE,2024-04-19,OPTIDX,PE,76700.0,0.05,NaN
29220,BFO,866815,10,BSXOPT,SENSEX2441976600PE,2024-04-19,OPTIDX,PE,76600.0,0.05,NaN
29221,BFO,867268,10,BSXOPT,SENSEX2441976500PE,2024-04-19,OPTIDX,PE,76500.0,0.05,NaN
29222,BFO,867344,10,BSXOPT,SENSEX2441976400PE,2024-04-19,OPTIDX,PE,76400.0,0.05,NaN
29223,BFO,867122,10,BSXOPT,SENSEX2441976300PE,2024-04-19,OPTIDX,PE,76300.0,0.05,NaN
29224,BFO,867207,10,BSXOPT,SENSEX2441976200PE,2024-04-19,OPTIDX,PE,76200.0,0.05,NaN
29225,BFO,866977,10,BSXOPT,SENSEX2441976100PE,2024-04-19,OPTIDX,PE,76100.0,0.05,NaN
29226,BFO,866781,10,BSXOPT,SENSEX2441976000PE,2024-04-19,OPTIDX,PE,76000.0,0.05,NaN


In [106]:
api.get_quotes(exchange='BFO', token='866735')

{'request_time': '22:23:33 15-04-2024',
 'stat': 'Ok',
 'exch': 'BFO',
 'tsym': 'SENSEX2441972000PE',
 'symname': 'BSXOPT',
 'seg': 'DER',
 'exd': '19-APR-2024',
 'instname': 'OPTIDX',
 'optt': 'PE',
 'pp': '2',
 'ls': '10',
 'ti': '0.05',
 'mult': '1',
 'lut': '1713199030',
 'uc': '746.45',
 'lc': '0.05',
 'oi': '31700',
 'cutof_all': 'false',
 'strprc': '72000.00',
 'prcftr_d': '(1 / 1 ) * (1 / 1)',
 'token': '866735',
 'lp': '104.00',
 'c': '17.55',
 'h': '104.00',
 'l': '104.00',
 'ap': '0.00',
 'o': '104.00',
 'v': '671510',
 'ltq': '0',
 'ltt': '00:00:00',
 'ltd': '15-04-2024',
 'tbq': '0',
 'tsq': '0',
 'bp1': '0.00',
 'sp1': '0.00',
 'bp2': '0.00',
 'sp2': '0.00',
 'bp3': '0.00',
 'sp3': '0.00',
 'bp4': '0.00',
 'sp4': '0.00',
 'bp5': '0.00',
 'sp5': '0.00',
 'bq1': '0',
 'sq1': '0',
 'bq2': '0',
 'sq2': '0',
 'bq3': '0',
 'sq3': '0',
 'bq4': '0',
 'sq4': '0',
 'bq5': '0',
 'sq5': '0',
 'bo1': '0',
 'so1': '0',
 'bo2': '0',
 'so2': '0',
 'bo3': '0',
 'so3': '0',
 'bo4': '0',
 '

In [107]:
api.get_quotes(exchange='NFO', token='67521')

{'request_time': '22:23:33 15-04-2024',
 'stat': 'Ok',
 'exch': 'NFO',
 'tsym': 'BANKNIFTY24APR24P47700',
 'cname': 'BANKNIFTY 24APR24 47700 PE ',
 'symname': 'BANKNIFTY',
 'seg': 'DER',
 'exd': '24-APR-2024',
 'instname': 'OPTIDX',
 'optt': 'PE',
 'pp': '2',
 'ls': '15',
 'ti': '0.05',
 'mult': '1',
 'lut': '1713180509',
 'uc': '1015.30',
 'lc': '0.05',
 'oi': '94905',
 'cutof_all': 'false',
 'strprc': '47700.00',
 'prcftr_d': '(1 / 1 ) * (1 / 1)',
 'token': '67521',
 'lp': '392.15',
 'c': '140.20',
 'h': '401.00',
 'l': '212.55',
 'ap': '315.99',
 'o': '239.90',
 'v': '690660',
 'ltq': '15',
 'ltt': '15:29:59',
 'ltd': '15-04-2024',
 'tbq': '11160',
 'tsq': '5235',
 'bp1': '392.25',
 'sp1': '398.70',
 'bp2': '392.00',
 'sp2': '399.00',
 'bp3': '391.10',
 'sp3': '400.00',
 'bp4': '391.00',
 'sp4': '405.00',
 'bp5': '390.30',
 'sp5': '410.00',
 'bq1': '45',
 'sq1': '15',
 'bq2': '195',
 'sq2': '990',
 'bq3': '15',
 'sq3': '150',
 'bq4': '30',
 'sq4': '30',
 'bq5': '390',
 'sq5': '780',

In [108]:
expiry_date
from datetime import datetime
date_obj = datetime.strptime(expiry_date, "%Y-%m-%d")
today = datetime.now()
diff = (date_obj - today).days
print(diff)

3


In [109]:
sell_premium = {
    "MIDCPNIFTY":[15,15,20,25,30],
    "FINNIFTY":[25,30,35,40,45],
    "NIFTY":[25,30,35,40,45],
    "BANKNIFTY": [50, 60,70,80,90,100],
    "SENSEX":[70,80,100,120,140,140,140,140]
}

In [110]:
buy_premium = {
    "MIDCPNIFTY":[2,2,3,3,3],
    "FINNIFTY":[3,4,4,5,5],
    "NIFTY":[3,4,4,5,5],
    "BANKNIFTY": [5,10,10,15,15],
    "SENSEX":[5,10,15,15,15,15,15,15]
}

In [111]:
ltp1 = buy_premium[index][diff]
ltp = sell_premium[index][diff]

print(ltp1)
print(ltp)
strikeList = []
for i in Cedf.index:
    strikeInfo = Cedf.loc[i]
    res = api.get_quotes(exchange='BFO', token=str(strikeInfo.Token))
    res = {'tsym': res['tsym'], 'lp': float(res['c']),'lotSize':strikeInfo.LotSize,'token':res['token']}
    strikeList.append(res)
    
strikedf1 = pd.DataFrame(strikeList)
strikedf1.sort_values(by = 'lp',inplace = True)
strikedf1['diff'] = abs(strikedf1['lp'] - ltp1)
strikedf1.sort_values(by = 'diff',inplace = True)
ce_buy = strikedf1.iloc[0].tsym
print(ce_buy)

strikedf = pd.DataFrame(strikeList)
strikedf.sort_values(by = 'lp',inplace = True)
strikedf['diff'] = abs(strikedf['lp'] - ltp)
strikedf.sort_values(by = 'diff',inplace = True)
ce_sell = strikedf.iloc[0].tsym
print(ce_sell)
ce_sl = 2*(strikedf.iloc[0].lp)
print(ce_sl)

15
120
SENSEX2441976800CE
SENSEX2441975400CE
253.9


In [112]:
ltp = buy_premium[index][diff]
ltp1 = sell_premium[index][diff]

print(ltp)
print(ltp1)

strikeList = []
for i in Pedf.index:
    strikeInfo = Pedf.loc[i]
    res = api.get_quotes(exchange='BFO', token=str(strikeInfo.Token))
    res = {'tsym': res['tsym'], 'lp': float(res['c']),'lotSize':strikeInfo.LotSize,'token':res['token']}
    strikeList.append(res)
strikedf = pd.DataFrame(strikeList)
strikedf.sort_values(by = 'lp',inplace = True)
strikedf['diff'] = abs(strikedf['lp'] - ltp)
strikedf.sort_values(by = 'diff',inplace = True)
pe_buy = strikedf.iloc[0].tsym
print(pe_buy)

strikedf1 = pd.DataFrame(strikeList)
strikedf1.sort_values(by = 'lp',inplace = True)
strikedf1['diff'] = abs(strikedf['lp'] - ltp1)
strikedf1.sort_values(by = 'diff',inplace = True)
pe_sell = strikedf1.iloc[0].tsym
print(pe_sell)
pe_sl = 2*(strikedf1.iloc[0].lp)
print(pe_sl)

15
120
SENSEX2441972100PE
SENSEX2441973200PE
232.1


In [113]:
# ltp = sell_premium[index][diff]
# print(ltp)

# strikeList = []
# for i in Cedf.index:
#     strikeInfo = Cedf.loc[i]
#     res = api.get_quotes(exchange='NFO', token=str(strikeInfo.Token))
#     res = {'tsym': res['tsym'], 'lp': float(res['lp']),'lotSize':strikeInfo.LotSize,'token':res['token']}
#     strikeList.append(res)
# strikedf = pd.DataFrame(strikeList)
# strikedf.sort_values(by = 'lp',inplace = True)
# strikedf['diff'] = abs(strikedf['lp'] - ltp)
# strikedf.sort_values(by = 'diff',inplace = True)
# ce_sell = strikedf.iloc[0].tsym

In [114]:
# ltp = sell_premium[index][diff]
# print(ltp)

# strikeList = []
# for i in Pedf.index:
#     strikeInfo = Pedf.loc[i]
#     res = api.get_quotes(exchange='NFO', token=str(strikeInfo.Token))
#     res = {'tsym': res['tsym'], 'lp': float(res['lp']),'lotSize':strikeInfo.LotSize,'token':res['token']}
#     strikeList.append(res)
# strikedf = pd.DataFrame(strikeList)
# strikedf.sort_values(by = 'lp',inplace = True)
# strikedf['diff'] = abs(strikedf['lp'] - ltp)
# strikedf.sort_values(by = 'diff',inplace = True)
# pe_sell = strikedf.iloc[0].tsym

In [115]:
print(ce_buy)
print(pe_buy)
print(ce_sell)
print(pe_sell)

SENSEX2441976800CE
SENSEX2441972100PE
SENSEX2441975400CE
SENSEX2441973200PE


In [116]:
import re

def get_new_strike(original_string, change):
    match = re.search(r'\d+(?=[A-Z]*$)', original_string)

    if match:
        # Extracted number
        number = int(match.group())
        
        print(number)

        # Add the change to the number
        new_number = number + change

        # Replace the old number with the new number in the original string
        new_string = original_string[:match.start()] + str(new_number) + original_string[match.end():]
        
        print("Original string:", original_string)
        print("New string:", new_string)
        return new_string
    else:
        print("Strike not in proper format")
        return None


In [117]:
# call buy leg
ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=ce_buy, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']


'24041501147277'

In [118]:
# put buy leg
ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=pe_buy, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']


'24041501147279'

In [119]:
# call sell leg
ret = api.place_order(buy_or_sell='S', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']


'24041501147280'

In [120]:
# put sell leg
ret = api.place_order(buy_or_sell='S', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']


'24041501147281'

In [121]:
# call sl order
ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='SL-LMT', price=ce_sl+2, trigger_price=ce_sl,
                        retention='DAY', remarks='my_order_001')

ce_sl_orderid = ret['norenordno']

In [122]:
# put sl order
ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='SL-LMT', price=pe_sl+2, trigger_price=pe_sl,
                        retention='DAY', remarks='my_order_001')

pe_sl_orderid = ret['norenordno']

In [123]:
print(ce_sl_orderid)
print(pe_sl_orderid)

24041501147283
24041501147284


In [124]:
# call adj1
ret = api.cancel_order(orderno=ce_sl_orderid)

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

new_ce_strike = get_new_strike(ce_sell, -1*changes_in_strike[id])
ce_sell = new_ce_strike

ret = api.place_order(buy_or_sell='S', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='SL-LMT', price=ce_sl+2, trigger_price=ce_sl,
                        retention='DAY', remarks='my_order_001')

ce_sl_orderid = ret['norenordno']

https://api.shoonya.com/NorenWClientTP//CancelOrder
{"request_time":"22:23:56 15-04-2024","stat":"Ok","result":"24041501147283"}
2441975400
Original string: SENSEX2441975400CE
New string: SENSEX2441975300CE


In [125]:
# call adj2
ret = api.cancel_order(orderno=ce_sl_orderid)

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

new_ce_strike = get_new_strike(ce_sell, -1*changes_in_strike[id])
ce_sell = new_ce_strike

ret = api.place_order(buy_or_sell='S', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='SL-LMT', price=ce_sl+2, trigger_price=ce_sl,
                        retention='DAY', remarks='my_order_001')

ce_sl_orderid = ret['norenordno']

https://api.shoonya.com/NorenWClientTP//CancelOrder
{"request_time":"22:23:57 15-04-2024","stat":"Ok","result":"24041501147289"}
2441975300
Original string: SENSEX2441975300CE
New string: SENSEX2441975200CE


In [126]:
# call adj3
ret = api.cancel_order(orderno=ce_sl_orderid)

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

new_ce_strike = get_new_strike(ce_sell, -1*changes_in_strike[id])
ce_sell = new_ce_strike

ret = api.place_order(buy_or_sell='S', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='SL-LMT', price=ce_sl+2, trigger_price=ce_sl,
                        retention='DAY', remarks='my_order_001')

ce_sl_orderid = ret['norenordno']

https://api.shoonya.com/NorenWClientTP//CancelOrder
{"request_time":"22:23:58 15-04-2024","stat":"Ok","result":"24041501147292"}
2441975200
Original string: SENSEX2441975200CE
New string: SENSEX2441975100CE


In [127]:
# call adj4
ret = api.cancel_order(orderno=ce_sl_orderid)

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

new_ce_strike = get_new_strike(ce_sell, -1*changes_in_strike[id])
ce_sell = new_ce_strike

ret = api.place_order(buy_or_sell='S', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=ce_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='SL-LMT', price=ce_sl+2, trigger_price=ce_sl,
                        retention='DAY', remarks='my_order_001')

ce_sl_orderid = ret['norenordno']

https://api.shoonya.com/NorenWClientTP//CancelOrder
{"request_time":"22:23:59 15-04-2024","stat":"Ok","result":"24041501147297"}
2441975100
Original string: SENSEX2441975100CE
New string: SENSEX2441975000CE


In [128]:
# put adj1
ret = api.cancel_order(orderno=pe_sl_orderid)

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

new_pe_strike = get_new_strike(pe_sell, changes_in_strike[id])
pe_sell = new_pe_strike

ret = api.place_order(buy_or_sell='S', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='SL-LMT', price=pe_sl+2, trigger_price=pe_sl,
                        retention='DAY', remarks='my_order_001')

pe_sl_orderid = ret['norenordno']

https://api.shoonya.com/NorenWClientTP//CancelOrder
{"request_time":"22:23:59 15-04-2024","stat":"Ok","result":"24041501147284"}
2441973200
Original string: SENSEX2441973200PE
New string: SENSEX2441973300PE


In [129]:
# put adj2
ret = api.cancel_order(orderno=pe_sl_orderid)

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

new_pe_strike = get_new_strike(pe_sell, changes_in_strike[id])
pe_sell = new_pe_strike

ret = api.place_order(buy_or_sell='S', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='SL-LMT', price=pe_sl+2, trigger_price=pe_sl,
                        retention='DAY', remarks='my_order_001')

pe_sl_orderid = ret['norenordno']

https://api.shoonya.com/NorenWClientTP//CancelOrder
{"request_time":"22:24:00 15-04-2024","stat":"Ok","result":"24041501147304"}
2441973300
Original string: SENSEX2441973300PE
New string: SENSEX2441973400PE


In [130]:
# put adj3
ret = api.cancel_order(orderno=pe_sl_orderid)

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

new_pe_strike = get_new_strike(pe_sell, changes_in_strike[id])
pe_sell = new_pe_strike

ret = api.place_order(buy_or_sell='S', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='SL-LMT', price=pe_sl+2, trigger_price=pe_sl,
                        retention='DAY', remarks='my_order_001')

pe_sl_orderid = ret['norenordno']

https://api.shoonya.com/NorenWClientTP//CancelOrder
{"request_time":"22:24:01 15-04-2024","stat":"Ok","result":"24041501147308"}
2441973400
Original string: SENSEX2441973400PE
New string: SENSEX2441973500PE


In [131]:
# put adj4
ret = api.cancel_order(orderno=pe_sl_orderid)

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

new_pe_strike = get_new_strike(pe_sell, changes_in_strike[id])
pe_sell = new_pe_strike

ret = api.place_order(buy_or_sell='S', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='MKT',
                        retention='DAY', remarks='my_order_001')

ret['norenordno']

ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='BFO', tradingsymbol=pe_sell, 
                        quantity=lots*lot_size, discloseqty=0,price_type='SL-LMT', price=pe_sl+2, trigger_price=pe_sl,
                        retention='DAY', remarks='my_order_001')

pe_sl_orderid = ret['norenordno']

https://api.shoonya.com/NorenWClientTP//CancelOrder
{"request_time":"22:24:03 15-04-2024","stat":"Ok","result":"24041501147313"}
2441973500
Original string: SENSEX2441973500PE
New string: SENSEX2441973600PE
